In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import signatory
import torch
from signax.signature import signature, signature_to_logsignature, signature_combine
from signax.module import SignatureTransform

In [ ]:
np.random.seed(0)

n_batch = 100
length = 100
dim = 4
x = np.random.rand(n_batch, length, dim)

depth = 3

In [ ]:
def jax_grad_over_sum(_callable):
    _callable = jax.jit(jax.grad(_callable))
    def wrap(*args, **kwargs):
        def _batch_call(_args, _kwargs):
            return jnp.sum(jax.vmap(_callable)(*_args, **_kwargs))

        return jax.value_and_grad(_batch_call)(args, kwargs)

    return wrap

In [ ]:
@jax_grad_over_sum
def signature_loss(path):
    sig = signature(path, depth)
    return sum(map(jnp.sum, sig))

# Compile function
_ = signature_loss(x)

In [ ]:
%%timeit
value, (grad,) = signature_loss(x)
value.block_until_ready()
grad.block_until_ready()

In [ ]:
torch_x = torch.as_tensor(x).requires_grad_(True)

In [ ]:
def torch_fun(x):
    sig = signatory.signature(x, depth)
    loss = torch.sum(sig)
    loss.backward()
    return loss

In [ ]:
%%timeit
loss = torch_fun(torch_x)

In [ ]:
dim = 3

input_ = [
    jnp.ones((dim,)),
    jnp.ones((dim, dim)),
    jnp.ones((dim, dim, dim)),
]

output = signature_to_logsignature(input_)
print(output)

In [ ]:
input_ = torch.ones((1, dim + dim * dim + dim * dim * dim))
log_signature = signatory.signature_to_logsignature(
    signature=input_, depth=3, channels=dim
)
print(log_signature)

In [ ]:
x1 = np.random.rand(n_batch, length, dim)
x2 = np.hstack((
    x1[:, -1:, :].copy(),
    np.random.randn(n_batch, length - 1, dim)
))


def batch_signature(path):
    return jax.vmap(lambda x_in: signature(x_in, depth))(path)


sig_x1 = batch_signature(x1)
sig_x2 = batch_signature(x2)

In [ ]:
def combine(signatures):
    return jax.vmap(signature_combine)(signatures)


# Compile function
_ = combine((sig_x1, sig_x2))

In [ ]:
%%timeit
combine((sig_x1, sig_x2))

In [ ]:
torch_x1 = torch.as_tensor(x1.copy()).requires_grad_(True)
torch_x2 = torch.as_tensor(x2.copy()).requires_grad_(True)

sig_torch_x1 = signatory.signature(torch_x1, depth)
sig_torch_x2 = signatory.signature(torch_x2, depth)

sig_torch_x1.retain_grad()
sig_torch_x2.retain_grad()

In [ ]:
def torch_combine(sig1, sig2):
    return signatory.signature_combine(sig1, sig2, dim, depth)


_ = torch_combine(sig_torch_x1, sig_torch_x2)

In [ ]:
%%timeit
torch_combine(sig_torch_x1, sig_torch_x2)